In [0]:
%%sh
# ["app-cross-domain", "basic", "cross-app", "cross-domain", "deduplicated", "domain-browser", "domain-outbound", "domain-referral", "domain-retention", "ranking"]
# aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.basic.v1/fact/granularity=m/month=201901/date=2019-01-31/device_code=ap/country_code=de/
 
 
# aws s3 cp s3://b2c-prod-data-pipeline-qa/aa.mobile.web/part-00000-d60e1650-58f6-4ebd-9384-16a0aa5bd621-c000.snappy.parquet - | head
# aws s3 cp s3://b2c-prod-data-pipeline-qa/aa.mobile.web/part-00000-6249f5d9-12b6-4ded-babc-30e9ee19281f-c000.snappy.parquet - | head
# aws s3 ls s3://b2c-prod-data-pipeline-qa/aa.mobile.web/ | wc -l


# aws s3 cp s3://b2c-prod-data-pipeline-qa/aa.mobile.web/_delta_log/._last_checkpoint.74f5dde8-976f-4074-a59d-4f0771417991.tmp - | head
# aws s3 ls s3://b2c-prod-dca-mobile-web-to-int/oss/


# --recursive --human --summarize | tail -30
# basic
# deduplicated
# ranking

# aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.app-cross-domain.v3/fact/granularity=m/month=202002/date=2020-02-29/device_code=ip/country_code=au/

# aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.basic.v3/fact/granularity=m/month=202002/date=2020-02-29/device_code=ip/country_code=au/

# aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v3/fact/granularity=m/month=202002/date=2020-02-29/device_code=ap/country_code=au/part-00000-231f5366-7613-44db-b5dc-f9f2196e651d.c000.gz.parquet
 
 
# aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/mobileweb.domain.v1/dimension/date=2019-03-15/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/mobileweb.domain.v2/dimension/
# aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/mobileweb.domain.v4/dimension/date=2019-03-15/

# aws s3 cp s3://b2c-prod-data-pipeline-unified-mobileweb-free/unified/temp/2018-11-13.deflate - | head
aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v3/fact/granularity=d/
aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobileweb.ranking.v4/fact/granularity=d/

 


In [0]:

# do not save version column, its not raw data value

# raw data:
print spark.read.parquet("s3://b2c-prod-dca-mobile-web-to-int/oss/MOBILE_WEB_DOMAIN_BASIC_METRICS/version=v1.0.0/granularity=DAY/date=2019-05-31/").show(1)

# unified data:
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.basic.v1/fact/granularity=d/month=201908/date=2019-08-01/device_code=ap/").printSchema()
print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.basic.v1/fact/").filter("version is not null").cache().groupBy("version").agg({"*":"count"}).show()
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.basic.v1/fact/granularity=d/month=201908/date=2019-08-23/device_code=ap/").printSchema() # have version
spark.read.parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.basic.v1/fact/granularity=d/month=201908/date=2019-08-25/device_code=ap/").printSchema()  # don't have version



In [0]:

ddf = spark.read.parquet("s3://b2c-prod-data-pipeline-qa/aa.mobile.web/").cache()
new_df =  ddf.filter("count1 != count2 and date != '2020-01-31' and date !='2020-02-29' ").orderBy("pack","granularity","date").select("pack","granularity","date","device_code").distinct()
# ddf.filter("device_code='ap' and pack='app-cross-domain'").show()
print [row.asDict() for row in new_df.collect()]




In [0]:

from datetime import datetime
from calendar import monthrange
from pyspark.sql.utils import AnalysisException

# date_list=set()
# for m in range(1,13):
#     month=str('%02d' % m)
#     date_list.add("2019-%s-%s" % (month ,monthrange(2019, m)[1]))

days="2020-02-29"
ip_packs_single_day = ["app-cross-domain", "basic", "cross-app", "cross-domain", "deduplicated", "domain-browser", "domain-outbound", "domain-referral", "domain-retention", "ranking"]
ap_packs_single_day = ["basic", "cross-domain", "deduplicated", "domain-browser", "domain-outbound", "domain-referral", "domain-retention", "ranking"]
up_packs_single_day = ["ranking"]
partitions_single_day = [(pack, 'm','ap', days) for pack in ap_packs_single_day]
partitions_single_day.extend([(pack, 'm','ip',days) for pack in ip_packs_single_day])
partitions_single_day.extend([(pack, 'm','up',days) for pack in up_packs_single_day])
# print partitions
# mobile-web.deduplicated.v1 w
# mobile-web.ranking.v1 w d
# mobile-web.basic.v1 w,d

print partitions_single_day[0]

for pack, granularity, device_code, date in partitions_single_day:
    # print '========================================== %s ========================================== start to test' % pack 
    month = datetime.strptime(date, "%Y-%m-%d").strftime('%Y%m')
    df1 = None
    df2 = None
    try:
        # print "%s, %s, %s, %s, %s started" % (pack, granularity, month, date, device_code)
        df2 = spark.read.option("basePath", 's3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.%s.v1/fact' % pack).parquet('s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.%s.v1/fact/granularity=%s/month=%s/date=%s/device_code=%s' % (pack, granularity, month, date, device_code))
        df1 = spark.read.format('delta').load('s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.%s.v000/fact' % pack).where("granularity='%s' and month='%s' and date='%s' and device_code='%s'" % ( granularity, month, date, device_code)).withColumnRenamed("est_browser_traffic_share","TS").withColumnRenamed("est_outbound_traffic_share","TS").withColumnRenamed("est_referral_traffic_share","TS")
        if sorted(df1.columns) != sorted(df2.columns):
            print "columns are different in %s, %s"  % (granularity, date)
            print sorted(df1.columns)
            print sorted(df2.columns)
            continue
        df1 = df1.select(df1.columns).drop('_identifier').cache()
        df2 = df2.select(df1.columns).drop('_identifier').cache()
        count1 = df1.exceptAll(df2).count()
        count2 = df2.exceptAll(df1).count()
        if count1 != 0 or count2 !=0:
            print "Count1 in %s, %s, %s, %s, %s: %s"  % (pack, granularity, month, date, device_code, count1)
            print df1.exceptAll(df2).take(1)
            print "Count2 in %s, %s, %s, %s, %s: %s"  % (pack, granularity, month, date, device_code, count2)
            print df2.exceptAll(df1).take(1)
            print "df1: %s df2: %s" % (df1.count(), df2.count())
            continue
        # result_df = spark.createDataFrame([(pack, granularity, month, date, device_code, df1.count())], schema=["pack","granularity","month","date","device_code","count"])
        # result_df.coalesce(1).write.format("delta").mode("append").partitionBy("event_month").save("s3://b2c-prod-data-pipeline-qa/aa.mobile.web/")
        print "%s, %s, %s, %s, %s pass" % (pack, granularity, month, date, device_code)
    except AnalysisException as e:
        print e
    finally:
        if df1:
            df1.unpersist()
        if df2:
            df2.unpersist()


In [0]:
%%sh
# "\nSELECT domain_id\nFROM plproxy.execute_select_nestloop($proxy$\n    SELECT domain_id, MAX(est_usage_penetration)::REAL AS est_usage_penetration\n    FROM mw.category_m_up_us\n    WHERE date BETWEEN '2020-02-01' AND '2020-02-29' AND\n        category_id = 800019 AND\n        rank_est_usage_penetration <= 3000\n    GROUP BY domain_id\n    ORDER BY MAX(est_usage_penetration) DESC\n    LIMIT 3000\n$proxy$) t (domain_id BIGINT, est_usage_penetration REAL)\nGROUP BY domain_id\nORDER BY MAX(est_usage_penetration) DESC, domain_id ASC\nLIMIT 3000\n"



PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 

-- select domain_id,est_usage_penetration from plproxy.execute_select_nestloop(\$proxy\$SELECT domain_id, MAX(est_usage_penetration)::REAL AS est_usage_penetration  FROM mw.category_m_up_us WHERE date BETWEEN '2020-02-01' -- AND '2020-02-29' and category_id = 800000 and rank_est_usage_penetration <= 3000 GROUP BY domain_id  ORDER BY MAX(est_usage_penetration) asc limit 3000 \$proxy\$)t (domain_id bigint , est_usage_penetration real) --limit 3000;

-- select sum(cnt) from plproxy.execute_select_nestloop(\$proxy\$SELECT count(1) as cnt  FROM mw.category_m_up_us WHERE date BETWEEN '2020-02-01' AND '2020-02-29' and category_id = 800000 and rank_est_usage_penetration <= -- 3000 GROUP BY domain_id  ORDER BY MAX(est_usage_penetration) asc \$proxy\$)t (cnt bigint) ;


select rank_est_usage_penetration, device_code, category_id,est_usage_penetration from plproxy.execute_select_nestloop(\$proxy\$SELECT device_code, category_id , category_id, est_usage_penetration from mw.category_m where domain_id = 70010000002035 and date = '2020-02-29' and category_id='800019' and country_code='us' limit 10  \$proxy\$)t (device_code char, rank_est_usage_penetration int,  category_id int  , est_usage_penetration real)  limit 10;

EOF


In [0]:
%%sh

# aws s3 cp s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v000/fact/granularity=m/month=202002/date=2020-02-29/  s3://b2c-prod-data-pipeline-qa/mobile_web/rank_v1/ --recursive
# PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 

#  select rank_est_usage_penetration , device_code ,category_id, est_usage_penetration from plproxy.execute_select_nestloop(\$proxy\$select device_code , rank_est_usage_penetration, category_id , est_usage_penetration
#                     from mw.category_m where  domain_id = 70010000004276 and date = '2020-02-29' and category_id= '800000' and country_code='us' limit 15 \$proxy\$)t (
#                     device_code char , rank_est_usage_penetration int, category_id int, est_usage_penetration real)  limit 10;
# EOF

# aws s3 cp s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v000/fact/granularity=m/month=202002/date=2020-02-29/  s3://b2c-prod-data-pipeline-qa/mobile_web/rank_v1/ --recursive

# aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v000/fact/granularity=m/month=202002/date=2020-02-29/device_code=up/country_code=au/
 




 
 
app-cross-domain", "basic", "cross-app", "cross-domain", "deduplicated", "domain-browser", "domain-outbound", "domain-referral", "domain-retention", "ranking


In [0]:

print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.app-cross-domain.v3/fact/granularity=m/month=202002/date=2020-02-29/").count()
print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.basic.v3/fact/granularity=m/month=202002/date=2020-02-29/").count()
print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.cross.v3/fact/granularity=m/month=202002/date=2020-02-29/").count()
print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.cross-domain.v3/fact/granularity=m/month=202002/date=2020-02-29/").count()
print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.deduplicated.v3/fact/granularity=m/month=202002/date=2020-02-29/").count()
print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.domain-browser.v3/fact/granularity=m/month=202002/date=2020-02-29/").count()
print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.domain-outbound.v3/fact/granularity=m/month=202002/date=2020-02-29/").count()
print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.domain-referral.v3/fact/granularity=m/month=202002/date=2020-02-29/").count()
print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.domain-retention.v3/fact/granularity=m/month=202002/date=2020-02-29/").count()
print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v3/fact/granularity=m/month=202002/date=2020-02-29/").count()

In [0]:


df_1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v1/fact/granularity=m/month=202002/date=2020-02-29/").filter(" country_code='us' and device_code='up' and category_id ='800000' ").withColumnRenamed("domain_id","wrong_domain_id").withColumnRenamed("est_usage_penetration","wrong_est_usage_penetration").withColumnRenamed("rank_est_usage_penetration","df1_rank_est_usage_penetration").select("df1_rank_est_usage_penetration","wrong_domain_id","wrong_est_usage_penetration")

df_2 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v3/fact/").where("granularity='m' and month='202002' and date='2020-02-29' and _identifier like '120200325%' ").filter(" category_id ='800000' and  country_code='us' and device_code='up'")

join_df = df_1.join(df_2, df_1.df1_rank_est_usage_penetration == df_2.rank_est_usage_penetration, "outer")
join_df.drop("_identifier").select("df1_rank_est_usage_penetration", "rank_est_usage_penetration","wrong_est_usage_penetration","est_usage_penetration","wrong_domain_id","domain_id", "category_id").orderBy("rank_est_usage_penetration").show(100000)


In [0]:


# "SELECT domain_id FROM plproxy.execute_select_nestloop($proxy$SELECT domain_id, MAX(est_usage_penetration)::REAL AS est_usage_penetration FROM mw.category_m_up_us WHERE date BETWEEN '2020-02-01' AND '2020-02-29' AND category_id = 800019 AND       rank_est_usage_penetration <= 3000  GROUP BY domain_id ORDER BY MAX(est_usage_penetration) DESC LIMIT 3000$proxy$) t (domain_id BIGINT, est_usage_penetration REAL)GROUP BY domain_id ORDER BY MAX(est_usage_penetration) DESC, domain_id ASC LIMIT 3000"
df_1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v1/fact/granularity=m/month=202002/date=2020-02-29/").filter(" country_code='us' and device_code='up' and category_id = '800000'").withColumnRenamed("domain_id","wrong_domain_id").withColumnRenamed("est_usage_penetration","wrong_est_usage_penetration").withColumnRenamed("rank_est_usage_penetration","df1_rank_est_usage_penetration").select("df1_rank_est_usage_penetration","wrong_domain_id","wrong_est_usage_penetration")

df_2 = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v3/fact/").where("granularity='m' and month='202002' and date='2020-02-29' and _identifier like '120200320%' ").filter("  country_code='us' and device_code='up' and category_id = '800000'")


join_df = df_1.join(df_2, df_1.wrong_domain_id == df_2.domain_id, "outer")
# join_df.show()
join_df.drop("_identifier").select("df1_rank_est_usage_penetration", "rank_est_usage_penetration","wrong_est_usage_penetration","est_usage_penetration","wrong_domain_id","domain_id", "category_id").filter("wrong_domain_id is null or domain_id is null").orderBy("rank_est_usage_penetration","df1_rank_est_usage_penetration", ascending=True).show(100000)
d1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v1/fact/granularity=m/month=202002/date=2020-02-29/").filter("category_id = '800019' and domain_id='70010000000232' and country_code='us'").show(5)




In [0]:


chart_df = spark.read.option('basePath','s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v1/fact/granularity=d/').parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v1/fact/granularity=m/month=*/date=*/device_code=up/country_code=us/").filter("category_id = '800019' and domain_id in ('70010000332425' ) ")


# chart_correct_df = spark.read.format("delta").load('s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v3/fact/').where("granularity = 'm' and device_code='up' and country_code='us'").filter("category_id = '800019' and domain_id in ('70010000003567' ) ")



l1=chart_df.collect()


new_list = [{ "date":x["date"], "rank":x["rank_est_usage_penetration"]} for x in l1 ]
print new_list[0]
print new_list[0]["date"]
print new_list[0]["rank"]

# print l[2]
# print "%table {}\t{}\t".format( "rank_est_usage_penetration" , "date" )
# for rank in l :
#     print "{}\t{}\t".format(rank["rank_est_usage_penetration"], rank["date"])
new_list.sort(reverse=True)
print "%table {}\t{}\t".format( "date" , "est_rank" )
for record in new_list :
    print "{}\t{}\t".format(record["date"],record["rank"])



In [0]:

from aadatapipelinecore.core.log import logger

import unittest
from pyspark.sql.utils import AnalysisException
from aadatapipelinecore.core.fs.device import s3
from aadatapipelinecore.core.fs import Conf

# s3_bucket_list = s3.S3Bucket(Conf(bucket_name='b2c-prod-data-pipeline-unified-mobileweb-paid'))
# path_list = s3_bucket_list.all(prefix="unified/mobile-web.basic.v1/fact/granularity=d/month=202002/", depth_is_1=True)
# print path_list[0]

[("app-cross-domain","MOBILE_WEB_APP_CROSS_DOMAIN_METRICS"), ("basic","MOBILE_WEB_DOMAIN_BASIC_METRICS"), ("cross-app","MOBILE_WEB_CROSS_APP_METRICS"), ("cross-domain","MOBILE_WEB_CROSS_DOMAIN_METRICS"), ("deduplicated","MOBILE_WEB_UNIFIED_PRODUCT_METRICS"), ("domain-browser","MOBILE_WEB_DOMAIN_BROWSER_METRICS"), ("domain-outbound","MOBILE_WEB_DOMAIN_OUTBOUND_METRICS"), ("domain-referral","MOBILE_WEB_DOMAIN_REFERRAL_METRICS"), ("domain-retention","MOBILE_WEB_DOMAIN_RETENTION_METRICS")]

data_source={"mobile-web.basic", "mobile-web.ranking","mobile-web.deduplicated"}
# data_source={"mobile-web.cross-app","mobile-web.deduplicated","mobile-web.domain-browser","mobile-web.domain-outbound","mobile-web.domain-referral","mobile-web.domain-retention","mobile-web.ranking"}
# data_source={"mobile-web.basic"}
# "mobile-web.app-cross-domain","mobile-web.basic"

def compare_count(granularity,namespace,date):
    try:
        print granularity,namespace,date
        df_old = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/{}.v000/fact/granularity={}/month={}/date={}/".format(namespace,granularity, date.replace("-","")[:-2], date))
        df_new = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/{}.v3/fact/".format(namespace)).where( "_identifier like '120200325%'").where("granularity='{}' and month='{}' and date='{}'".format(granularity, date.replace("-","")[:-2], date)).cache()
        df_old = df_old.select(df_old.columns).drop('_identifier').cache()
        df_new = df_new.select(df_new.columns).drop('_identifier').drop("date").drop("granularity").drop("month").cache()
        c1 = df_new.count()
        c2 = df_old.count()
        # df_new.createOrReplaceTempView("new")
        # df_old.createOrReplaceTempView("old")
        # spark.sql("select app_id, cross_domain_id from (select app_id, cross_domain_id from old EXCEPT select app_id, cross_domain_id from new) as prod").show()
        if c1 != c2 :
            print "not equal!!! namespace: {} , new count is {} , old count is {} diff is {} ".format(namespace,  c1, c2 , c1-c2)
        else:
            print "namespace {} pass".format(namespace)
            
    except AnalysisException as e:
        print e
    finally:
        if df_old:
            df_old.unpersist()
        if df_new:
            df_new.unpersist()

for ds in data_source:
    compare_count("m", ds, "2020-02-29")

In [0]:
%%sh

# aws s3 ls s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.basic.v1/fact/granularity=d/month=202003/date=2020-03-08/device_code=ap/country_code=us/

 
 

PGPASSWORD='2mHdFW6%#REu' psql -h internal-aa-prod-plproxy-internal-2-31838298.us-east-1.elb.amazonaws.com -U app_bdp_usage_qa -d cohort -p 7432 << EOF 

-- select sum(cnt) from plproxy.execute_select(\$proxy\$select count(1) as cnt 
--                    from mw.domain_basic_d where device_code = 'ap' and
--                         country_code='us' and date='2020-03-08' limit 5 \$proxy\$)t (cnt bigint)  limit 3;

-- select sum(cnt) from plproxy.execute_select(\$proxy\$select count(1) as cnt 
--                   from mw.domain_basic_d where  date='2020-03-08' limit 5 \$proxy\$)t (cnt bigint)  limit 3;
                   

-- select sum(cnt) from plproxy.execute_select(\$proxy\$select count(1) as cnt 
--                   from mw.category_d where  date='2020-03-08' limit 5 \$proxy\$)t (cnt bigint)  limit 3;

select domain_id, category_id, est_usage_penetration , rank_est_usage_penetration, device_code, country_code
 from plproxy.execute_select(\$proxy\$select domain_id, category_id, est_usage_penetration , rank_est_usage_penetration ,device_code,country_code
                  from mw.category_d where  date='2020-03-08' and device_code='up' limit 5 \$proxy\$)t (domain_id bigint , category_id int , est_usage_penetration real, rank_est_usage_penetration int , device_code char, country_code char)  limit 3;



select domain_id, device_code, country_code, est_average_active_users, est_usage_penetration, est_total_time, est_average_session_per_user, est_average_session_duration, est_average_time_per_user
 from plproxy.execute_select(\$proxy\$select domain_id ,device_code,country_code, est_average_active_users, est_usage_penetration, est_total_time, est_average_session_per_user, est_average_session_duration, est_average_time_per_user  from mw.domain_basic_d where  date='2020-03-08' and device_code='ap' limit 5 \$proxy\$)t (domain_id bigint , device_code char, country_code char, est_average_active_users real, est_usage_penetration real, est_total_time real, est_average_session_per_user real , est_average_session_duration real, est_average_time_per_user real)  limit 3;



EOF



In [0]:


print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.basic.v1/fact/granularity=d/month=202003/date=2020-03-08/").filter("domain_id='70010000099289' and country_code='de' and device_code='ap' ").show()
print 'test'
print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.basic.v3/fact/").where("granularity='d' and month='202003' and date='2020-03-08'").filter("domain_id='70010000099289' and country_code='de' and device_code='ap' ").show()


# print spark.read.parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v1/fact/granularity=d/month=202003/date=2020-03-08/").filter("category_id='800000' and domain_id='70010000000004' and device_code='up' and country_code='de'").show()
# print 'test'
# print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v3/fact/").where("granularity='d' and month='202003' and date='2020-03-08'").filter("category_id='800000' and domain_id='70010000000004' and device_code='up' and country_code='de' ").show()

In [0]:


# cross_domain_id like '70010000001654%' and
# filter("app_id='100' and cross_domain_id like '70010000016986%' ").show()
d1 = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v000/fact/granularity=m/month=202002/date=2020-02-29/").drop("_identifier").orderBy("category_id","domain_id","rank_est_usage_penetration")

d2= spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.ranking.v3/fact/").where("granularity='m' and month='202002' and date='2020-02-29' and _identifier like '120200320%' ").drop("granularity","month","date","device_code","country_code").drop("_identifier").orderBy("category_id","domain_id","rank_est_usage_penetration")

print d1.count()
print d2.count()
# d2.subtract(d1).show()
# d1.subtract(d2).show()
d1 = d1.withColumnRenamed("category_id", "d1_category_id").withColumnRenamed("domain_id", "d1_domain_id")
d3 = d1.join(d2, [d1.d1_category_id == d2.category_id, d1.d1_domain_id==d2.domain_id], how = 'outer')


d3.filter("d1_category_id is null").groupBy("category_id").agg({"category_id":"count"}).show()
d3.filter("category_id is null").groupBy("device_code").agg({"device_code":"count"}).show()

# df3 = df1.join(df2, [df1.name == df2.name] , how = 'inner' )
# df3.filter(df3.df1_count != df3.df2_count).show()


In [0]:

from datetime import datetime
from calendar import monthrange
from pyspark.sql.utils import AnalysisException

date_list=set()
for m in range(1,3):
    month=str('%02d' % m)
    date_list.add("2020-%s-%s" % (month ,monthrange(2020, m)[1]))

ip_packs = ["basic"]
ap_packs = ["basic"]
up_packs = ["basic"]
partitions = [(pack, 'w','ap', days) for pack in ap_packs for days in date_list]
partitions.extend([(pack, 'w','ip',days) for pack in ip_packs for days in date_list])
partitions.extend([(pack, 'w','up',days) for pack in up_packs for days in date_list])
# print partitions
# mobile-web.deduplicated.v1 w
# mobile-web.ranking.v1 w d
# mobile-web.basic.v1 w,d



for pack, granularity, device_code, date in partitions:
    # print '========================================== %s ========================================== start to test' % pack 
    month = datetime.strptime(date, "%Y-%m-%d").strftime('%Y%m')
    df1 = None
    df2 = None
    try:
        # print "%s, %s, %s, %s, %s started" % (pack, granularity, month, date, device_code)
        df2 = spark.read.option("basePath", 's3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.%s.v1/fact' % pack).parquet('s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.%s.v1/fact/granularity=%s/month=%s/date=%s/device_code=%s' % (pack, granularity, month, date, device_code))
        df1 = spark.read.format('delta').load('s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.%s.v3/fact' % pack).where("granularity='%s' and month='%s' and date='%s' and device_code='%s'" % ( granularity, month, date, device_code)).withColumnRenamed("est_browser_traffic_share","TS").withColumnRenamed("est_outbound_traffic_share","TS").withColumnRenamed("est_referral_traffic_share","TS")
        if sorted(df1.columns) != sorted(df2.columns):
            print "columns are different in %s, %s"  % (granularity, date)
            print sorted(df1.columns)
            print sorted(df2.columns)
            continue
        df1 = df1.select(df1.columns).drop('_identifier').cache()
        df2 = df2.select(df1.columns).drop('_identifier').cache()
        count1 = df1.exceptAll(df2).count()
        count2 = df2.exceptAll(df1).count()
        if count1 != 0 or count2 !=0:
            print "Count1 in %s, %s, %s, %s, %s: %s"  % (pack, granularity, month, date, device_code, count1)
            print df1.exceptAll(df2).take(1)
            print "Count2 in %s, %s, %s, %s, %s: %s"  % (pack, granularity, month, date, device_code, count2)
            print df2.exceptAll(df1).take(1)
            print "df1: %s df2: %s" % (df1.count(), df2.count())
            continue
        result_df = spark.createDataFrame([(pack, granularity, month, date, device_code, df1.count())], schema=["pack","granularity","month","date","device_code","count"])
        result_df.coalesce(1).write.format("delta").mode("append").partitionBy("event_month").save("s3://b2c-prod-data-pipeline-qa/aa.mobile.web/")
        print "%s, %s, %s, %s, %s pass" % (pack, granularity, month, date, device_code)
    except AnalysisException as e:
        print e
    finally:
        if df1:
            df1.unpersist()
        if df2:
            df2.unpersist()


In [0]:
%%sh
PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.26.136 -U app_bdp_usage_qa -d aa -p 6432 << EOF 
--select * from plproxy.execute_select(\$proxy\$select date, app_id, city_id, est_average_active_users ,est_average_bytes_per_user
--               from mu.city_app_w where city_id='31248510' and app_id='20600000304614'  and country_code='ae' and date='2020-02-29'
--                \$proxy\$) t (date date, 
--               app_id bigint, city_id bigint, est_average_active_users real , est_average_bytes_per_user real) limit 3;
-- SELECT domain_id AS domain_id, name AS domain_name FROM aa_domain_metadata WHERE is_disabled IN ('f') and sensitive_status IN (0);
-- \d aa_domain_metadata
SELECT  domain_id , update_time  FROM aa_domain_metadata WHERE is_disabled IN ('f') and sensitive_status IN (0) and domain_id in ( '70010000208830' , '70010000210592', '70010000002208' , '70010000021237', '70010000000321', '70010000211341', '70010000001238');
SELECT  *  FROM aa_domain_metadata WHERE is_disabled IN ('f') and sensitive_status IN (0) limit 3


EOF


In [0]:

from pyspark.sql.utils import AnalysisException
import datetime
from calendar import monthrange
from datetime import date, timedelta
from datetime import datetime as dt

# date_list_month=set()
# for m in range(1,4):
#     month=str('%02d' % m)
#     date_list_month.add("2020-%s-%s" % (month ,monthrange(2020, m)[1]))
# for m in range(1,13):
#     month=str('%02d' % m)
#     date_list_month.add("2019-%s-%s" % (month ,monthrange(2019, m)[1]))

# sorted(date_list_month)
# ip_packs = ["app-cross-domain", "basic", "cross-app", "cross-domain", "deduplicated", "domain-browser", "domain-outbound", "domain-referral", "domain-retention", "ranking"]
# ap_packs = ["app-cross-domain", "basic", "cross-app", "cross-domain", "deduplicated", "domain-browser", "domain-outbound", "domain-referral", "domain-retention", "ranking"]
# up_packs = ["ranking"]
# partitions = [(pack, 'm','ap', days) for pack in ap_packs for days in date_list_month]
# partitions.extend([(pack, 'm','ip',days) for pack in ip_packs for days in date_list_month])
# partitions.extend([(pack, 'm','up',days) for pack in up_packs for days in date_list_month])
# print partitions
# special_month=set()
# for m in range(1,13):
#     month=str('%02d' % m)
#     special_month.add("2018-%s-%s" % (month ,monthrange(2019, m)[1]))
# new_partitions = [('cross-app', 'm','ip', days) for days in special_month]    


partitions=[]
start = '2020-03-01'
end = '2020-03-20'
real_date1 = datetime.date(*[int(x) for x in start.split('-')])
real_date2 = datetime.date(*[int(x) for x in end.split('-')])
date_range =  real_date2 - real_date1
dates = list()
for days in xrange(date_range.days):
    dates.append(real_date1 + datetime.timedelta(days))
dates.sort()
ip_packs_d = ["ranking", "basic"]
ap_packs_d = ["ranking", "basic"]
up_packs_d = ["ranking", "basic"]
partitions.extend([(pack, 'd','ap', str(days)) for pack in ip_packs_d for days in dates])
partitions.extend([(pack, 'd','ip',str(days)) for pack in ap_packs_d for days in dates])
partitions.extend([(pack, 'd','up',str(days)) for pack in up_packs_d for days in dates])


dstart = date(2019,03,01)
dend = date(2020,03,20)
# this will return all sundays between start-end dates.
week_days = [dstart + timedelta(days=x) for x in range((dend-dstart).days + 1) if (dstart + timedelta(days=x)).weekday() == 5]
sorted(week_days)
ip_packs_d = ["deduplicated", "basic","ranking"]
ap_packs_d = ["deduplicated", "basic","ranking"]
up_packs_d = ["deduplicated", "basic","ranking"]
partitions.extend([(pack, 'w','ap', str(days)) for pack in ip_packs_d for days in week_days])
partitions.extend([(pack, 'w','ip',str(days)) for pack in ap_packs_d for days in week_days])
partitions.extend([(pack, 'w','up',str(days)) for pack in up_packs_d for days in week_days])

partitions.sort()
# basic
# deduplicated
# ranking


for pack, granularity, device_code, date in partitions:
    # print '========================================== %s ========================================== start to test' % pack 
    month = dt.strptime(date, "%Y-%m-%d").strftime('%Y%m')
    df1 = None
    df2 = None
    try:
        # print "%s, %s, %s, %s, %s started" % (pack, granularity, month, date, device_code)
        df2 = spark.read.option("basePath", 's3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.%s.v1/fact' % pack).parquet('s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.%s.v1/fact/granularity=%s/month=%s/date=%s/device_code=%s' % (pack, granularity, month, date, device_code))
        df1 = spark.read.format('delta').load('s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.%s.v3/fact' % pack).where("granularity='%s' and month='%s' and date='%s' and device_code='%s'" % ( granularity, month, date, device_code)).withColumnRenamed("est_browser_traffic_share","TS").withColumnRenamed("est_outbound_traffic_share","TS").withColumnRenamed("est_referral_traffic_share","TS")
        df_columns = df2.columns
        if "version" in df_columns:
            df_columns.remove("version") 
        
        if sorted(df_columns) != sorted(df1.columns):
            print "columns are different in %s, %s"  % (granularity, date)
            print sorted(df1.columns)
            print sorted(df2.columns)
            continue
        df1 = df1.select(df1.columns).drop('_identifier').cache()
        df2 = df2.select(df1.columns).drop('_identifier').cache()
        count1 = df1.count()
        count2 = df2.count()
        count1_diff = df1.exceptAll(df2).count()
        count2_diff = df2.exceptAll(df1).count()
        if count1_diff != 0 or count1_diff !=0:
            print "Count1 in %s, %s, %s, %s, %s: %s"  % (pack, granularity, month, date, device_code, count1_diff)
            print df1.exceptAll(df2).take(1)
            print "Count2 in %s, %s, %s, %s, %s: %s"  % (pack, granularity, month, date, device_code, count2_diff)
            print df2.exceptAll(df1).take(1)
            print "df1: %s df2: %s" % (count1, count2)
            continue
        result_df = spark.createDataFrame([(pack, granularity, month, date, device_code,count1, count2)], schema=["pack","granularity","month","date","device_code","count1", "count2"])
        result_df.coalesce(1).write.format("delta").mode("append").partitionBy("event_month").save("s3://b2c-prod-data-pipeline-qa/aa.mobile.web/")
        # print "%s, %s, %s, %s, %s pass" % (pack, granularity, month, date, device_code)
    except AnalysisException as e:
        if "v3" in str(e):
            result_df = spark.createDataFrame([(pack, granularity, month, date, device_code,0, 0)], schema=["pack","granularity","month","date","device_code","count1", "count2"])
            result_df.coalesce(1).write.format("delta").mode("append").partitionBy("event_month").save("s3://b2c-prod-data-pipeline-qa/aa.mobile.web/")
            print 'data is not exist !!!! {},{}, {}, {} , {}, {}'.format(e, pack, granularity, month, date, device_code)
    finally:
        if df1:
            df1.unpersist()
        if df2:
            df2.unpersist()


In [0]:
%%sh
echo `date`

PGPASSWORD='2mHdFW6%#REu' psql -h 10.2.26.136 -U app_bdp_usage_qa -d dna -p 6432 << EOF 

SELECT * FROM in_domain_category_mapping order by last_updated desc limit 20
EOF 

In [0]:

import pyspark.sql.functions as F
# ip_packs_single_day = ["app-cross-domain",  "cross-app", "cross-domain", "deduplicated", "domain-browser", "domain-outbound", "domain-referral", "domain-retention", "ranking", "basic"]

def get_compared_data(date):
    df_new = spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.basic.v3/fact/").where("granularity='m' and month='{}' and date='{}' ".format(date.replace("-","")[:-2], date))
    df_old = spark.read.parquet("s3://b2c-prod-data-pipeline-unified-mobileweb-paid/unified/mobile-web.basic.v000/fact/granularity=m/month=202002/date={}/".format(date))

    list1= df_new.sample(False, 0.001, seed=0).limit(20).take(20)
    return list1, df_old

test_list, df_old = get_compared_data('2020-02-29')



def compare(test_list, df_old):
    columns_list = df_old.columns
    print columns_list
    columns_list.remove("_identifier")
    # columns_list.remove("TS")
    print len(test_list)
    for x in range(20):
        # list2 = df_old.filter("app_id='{}' and cross_domain_id='{}' ".format(test_list[x]["app_id"], test_list[x]["cross_domain_id"])).collect()
        # list2 = df_old.filter("domain_id='{}' ".format(test_list[x]["domain_id"])).collect()
        # list2 = df_old.filter("domain_id='{}' and cross_app_id='{}' ".format(test_list[x]["domain_id"],test_list[x]["cross_app_id"])).collect()
        # list2 = df_old.filter("domain_id='{}' and browser_app_id='{}' ".format(test_list[x]["domain_id"],test_list[x]["browser_app_id"])).collect()
        # list2 = df_old.filter("domain_id='{}' and outbound_domain_id='{}' ".format(test_list[x]["domain_id"],test_list[x]["outbound_domain_id"])).collect()
        # list2 = df_old.filter("domain_id='{}' and referral_domain_id='{}' ".format(test_list[x]["domain_id"],test_list[x]["referral_domain_id"])).collect()
        # list2 = df_old.filter("domain_id='{}'and retention_days='{}' ".format(test_list[x]["domain_id"],test_list[x]["retention_days"] )).collect()
        # list2 = df_old.filter("category_id='{}'and domain_id='{}' and device_code='{}' ".format(test_list[x]["category_id"],test_list[x]["domain_id"],test_list[x]["device_code"] )).collect()
        list2 = df_old.filter("domain_id='{}'  ".format(test_list[x]["domain_id"])).collect()

        for column in columns_list:
            # print column
            if test_list[x][column] != list2[0][column]:
                print 'not equal',column, test_list[x][column], list2[0][column]
    print 'Finished'           

compare(test_list, df_old)

# print spark.read.format("delta").load("s3://b2c-prod-data-pipeline-unified-usage/unified/city-level-usage.v2/fact/").where("granularity='m' and date='2020-02-29'").filter("device_code='ap'").count()